In [7]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [8]:
def extrair_anuncios(url_base, total_pages, nome_arquivo):
    # Configuração do Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Executa o navegador em modo headless
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Acessando a URL inicial
    driver.get(url_base)
    time.sleep(5)  # Espera carregar a página inicial

    # Declarando variável cards
    cards = []

    for current_page in range(1, total_pages + 1):
        # Obtendo o HTML da página atual
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Obtendo as Tags de interesse
        try:
            containers = soup.find_all('div', {'class': 'js-card-selector'})
            if not containers:
                containers = soup.find_all('article', {'class': 'property-card__container js-property-card'})
            if not containers:
                print(f'Erro ao processar a página {current_page}: container não encontrado')
                break
        except AttributeError:
            print(f'Erro ao processar a página {current_page}: estrutura da página mudou')
            break

        # Coletando as informações dos CARDS
        for container in containers:
            card = {}

            # Valor
            valor_p = container.find('div', {'class': 'property-card__price js-property-card-prices js-property-card__price-small'})
            if valor_p:
                valor_text = valor_p.get_text(strip=True).split('/')[0].strip()
                valor = valor_text.replace('R$', '').replace('.', '').replace(',', '')
                try:
                    card['valor'] = int(valor)
                except ValueError:
                    card['valor'] = 0
            else:
                card['valor'] = 0

            # Endereço
            endereco_tag = container.find('span', {'class': 'property-card__address'})
            card['endereco'] = endereco_tag.get_text(strip=True) if endereco_tag else 'Não disponível'
            
            # Quartos
            quartos = container.find('li', {'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'})
            card['quartos'] = quartos.find('span', {'class': 'property-card__detail-value js-property-card-value'}).get_text(strip=True) if quartos else 1

            # Garagem
            garagem = container.find('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'})
            card['garagem'] = garagem.find('span', {'class': 'property-card__detail-value js-property-card-value'}).get_text(strip=True) if garagem else 0

            # Banheiros
            banheiro = container.find('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'})
            card['banheiro'] = banheiro.find('span', {'class': 'property-card__detail-value js-property-card-value'}).get_text(strip=True) if banheiro else 1

            # Metragem
            metro = container.find('li', {'class': 'property-card__detail-item property-card__detail-area'})
            card['area(m2)'] = metro.find('span', {'class': 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area'}).get_text(strip=True) if metro else 30

            # IPTU e Condomínio
            condominio_tag = container.find('div', {'class': 'property-card__price-details--condo'})
            if condominio_tag:
                condominio_text = condominio_tag.get_text(strip=True).replace('Condomínio:', '').replace('R$', '').replace('.', '').replace(',', '').strip()
                try:
                    card['condominio'] = int(condominio_text)
                except ValueError:
                    card['condominio'] = 0
            else:
                card['condominio'] = 0
            
            cards.append(card)
        
        print(f"Número de containers encontrados na página {current_page}: {len(containers)}")
        
        # Tenta ir para a próxima página
        if current_page < total_pages:
            try:
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f'//button[@title="Página {current_page + 1}"]'))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                driver.execute_script("arguments[0].click();", next_page_button)
                time.sleep(5)  # Espera carregar a próxima página
            except Exception as e:
                print(f"Erro ao tentar ir para a próxima página: {e}")
                break

    # Fechando o navegador
    driver.quit()

    # Salvando os dados acumulados em um arquivo JSON
    if cards:
        try:
            with open(nome_arquivo, 'r', encoding='utf-8') as file:
                existing_data = json.load(file)
        except FileNotFoundError:
            existing_data = []

        existing_data.extend(cards)
        with open(nome_arquivo, 'w', encoding='utf-8') as file:
            json.dump(existing_data, file, ensure_ascii=False, indent=4)

    return len(cards)


In [9]:
url_ap = 'https://www.vivareal.com.br/aluguel/minas-gerais/belo-horizonte/cobertura_residencial'
extrair_anuncios(url_ap, total_pages=18, nome_arquivo='viva_cob.json')

Número de containers encontrados na página 1: 36
Número de containers encontrados na página 2: 36
Número de containers encontrados na página 3: 34
Número de containers encontrados na página 4: 36
Número de containers encontrados na página 5: 36
Número de containers encontrados na página 6: 34
Número de containers encontrados na página 7: 36
Número de containers encontrados na página 8: 36
Número de containers encontrados na página 9: 36
Número de containers encontrados na página 10: 35
Número de containers encontrados na página 11: 36
Número de containers encontrados na página 12: 36
Número de containers encontrados na página 13: 36
Número de containers encontrados na página 14: 35
Número de containers encontrados na página 15: 35
Número de containers encontrados na página 16: 35
Número de containers encontrados na página 17: 34
Número de containers encontrados na página 18: 36


638